In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import copy
import socket
import os
import sys
import pandas as pd
import scipy.io as sio
import matplotlib
import scipy.signal as sg
from scipy.io import wavfile
import math
import scipy as sp
import pylab
import h5py
import pickle
import seaborn as sns
import random
import collections
import logging
import datetime
import itertools
import tensorflow as tf
import tensorflow.keras as keras
import sklearn
import gc
import importlib
import librosa
import librosa.display
from glob import glob
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

sys.path.append("..") # Adds higher directory to python modules path.
from helpers.dataset_vars import *
from helpers.myclasses import *

In [2]:
from math_utils import *

# location of wavs

In [6]:
data_folder = os.path.abspath('/mnt/cube/mturvz/normed_motifs/all_motifs')

In [7]:
all_wavs = sorted(glob(os.path.join(data_folder, '*.wav')))

In [8]:
print(all_wavs[0])

/mnt/cube/mturvz/normed_motifs/all_motifs/A2-1_165_s_03_@1.wav


In [10]:
y, sr = librosa.load(all_wavs[0], sr=None)

In [11]:
spec_pars = {'fft_size': 2048, 
             'step_size': 256,
             'n_stars':int(hz2star(sr/2)),
             'fmin': 20}

# compile all

In [12]:
spec_file = os.path.join('/mnt/cube/kai/data_processed/others', 'Mike_stims_specs_star.p')
if os.path.exists(spec_file):
    paragraph_print(f'Skipping', separation = '>')
else:
    all_specs = list()
    all_indeces = list()
    all_names = list()
    for wav in all_wavs:
        wav_name=wav.split('/')[-1].split('.')[0]
        y, sr = librosa.load(wav, sr=None)
        S_star = star_spectrogram(y=y, sr=sr, n_stars=spec_pars['n_stars'], n_fft=spec_pars['fft_size'], 
                                      hop_length=spec_pars['step_size'], win_length=spec_pars['fft_size'], 
                                      window='hann')
        S_norm = librosa.util.normalize(librosa.power_to_db(S_star, ref=np.max))
        all_specs.append(S_norm)
        all_names.append(wav)
    morph_df = pd.DataFrame()
    morph_df['wav'] = all_names
    morph_df['spec'] = all_specs

In [13]:
morph_df.head()

,wav,spec
0,/mnt/cube/mturvz/normed_motifs/all_motifs/A2-1...,"[[-0.89071786, -0.9061238, -0.94669884, -0.995..."
1,/mnt/cube/mturvz/normed_motifs/all_motifs/A3_+...,"[[-0.8159339, -0.8329322, -0.8805124, -0.95064..."
2,/mnt/cube/mturvz/normed_motifs/all_motifs/A62_...,"[[-0.7634527, -0.77958375, -0.8326478, -0.9381..."
3,/mnt/cube/mturvz/normed_motifs/all_motifs/B11_...,"[[-0.9130305, -0.9239519, -0.95564747, -1.0, -..."
4,/mnt/cube/mturvz/normed_motifs/all_motifs/B12_...,"[[-0.8242189, -0.84699905, -0.93398845, -1.0, ..."


In [14]:
morph_df.to_pickle(spec_file)
del morph_df